# Data Interrogation

First of all - what's in this file?

In [2]:
%%bash
ls

Collaboration Points.ipynb
Collaboration_scores.ipynb
README.md
data
output_data


In [3]:
import pandas as pd

# File importation and cleaning

ipgod102.csv is our key file. This is the "Applicant summary" portion of IPGod.

In [4]:
df_ipgod102 = pd.read_csv("data/ipgod102.csv")

In [5]:
df_ipgod102.head()

,ipa_applt_id,australian_appl_no,appln_type,name,cleanname,country,australian,entity,ipa_id,abn,...,lon,qg,state,postcode,patstat_appln_id,applicant_type,sa2_main11,sa2_name11,gcc_code11,gcc_name11
0,6657148,1962018599,PAT,NON-ENTITY 6652,NON-ENTITY 6652,ZZ,NaN,1.0,415292.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN
1,6659287,1975086712,PAT,NON-ENTITY 599789,NON-ENTITY 599789,ZZ,NaN,1.0,484115.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN
2,6562627,1977029773,PAT,NON-ENTITY 612514,NON-ENTITY 612514,ZZ,NaN,1.0,455771.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN
3,281770,2003252584,PAT,Koninklijke BAM Groep N.V.,KONINKLIJKE BAM GROEP,NL,NaN,1.0,139265.0,NaN,...,NaN,NaN,NaN,NaN,2621618.0,International,NaN,NaN,NaN,NaN
4,282135,2003253837,PAT,General Electric Company,GENERAL ELECTRIC CO,US,NaN,1.0,77334.0,NaN,...,NaN,NaN,NaN,NaN,2622844.0,International,NaN,NaN,NaN,NaN


ipgod101.csv is the patent summary.

Read in ipgod101.csv to include the year of application.

In [6]:
df_ipgod101 = pd.read_csv("data/ipgod101.csv", index_col="australian_appl_no")
df_ipgod101.application_year[2003303556]

2007

We'll use this later on to work on a subset, rather than mapping it to the (enormous) IPGod102 file.

# Giving JSON a crack

The handsome and marvellous Macca has extracted data elsewhere that lists:
* Patents with multiple applicants, which
* Consist of multiple, non-non-entities

These are packaged into "data.json".

In [7]:
import json

In [8]:
with open("data/data.json",'r') as f:
    json_data = json.load(f)

In [9]:
json_data
print(len(json_data))

1915


We'll be using this in conjunction with IPGod102, so let's refresh ourselves on the structure of the data:

In [10]:
df_ipgod102.head()

,ipa_applt_id,australian_appl_no,appln_type,name,cleanname,country,australian,entity,ipa_id,abn,...,lon,qg,state,postcode,patstat_appln_id,applicant_type,sa2_main11,sa2_name11,gcc_code11,gcc_name11
0,6657148,1962018599,PAT,NON-ENTITY 6652,NON-ENTITY 6652,ZZ,NaN,1.0,415292.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN
1,6659287,1975086712,PAT,NON-ENTITY 599789,NON-ENTITY 599789,ZZ,NaN,1.0,484115.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN
2,6562627,1977029773,PAT,NON-ENTITY 612514,NON-ENTITY 612514,ZZ,NaN,1.0,455771.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN
3,281770,2003252584,PAT,Koninklijke BAM Groep N.V.,KONINKLIJKE BAM GROEP,NL,NaN,1.0,139265.0,NaN,...,NaN,NaN,NaN,NaN,2621618.0,International,NaN,NaN,NaN,NaN
4,282135,2003253837,PAT,General Electric Company,GENERAL ELECTRIC CO,US,NaN,1.0,77334.0,NaN,...,NaN,NaN,NaN,NaN,2622844.0,International,NaN,NaN,NaN,NaN


In [11]:
df_ipgod102[df_ipgod102['australian_appl_no'].isin(list(json_data.keys()))]

,ipa_applt_id,australian_appl_no,appln_type,name,cleanname,country,australian,entity,ipa_id,abn,...,lon,qg,state,postcode,patstat_appln_id,applicant_type,sa2_main11,sa2_name11,gcc_code11,gcc_name11


In [12]:
# Convert the keys to integers
collab_appl_nos = list(json_data.keys())
collab_appl_nos = [int(x) for x in collab_appl_nos]

Let's produce our subset of the collaborative patents.

In [13]:
df_collab_subset = df_ipgod102[df_ipgod102['australian_appl_no'].isin(collab_appl_nos)]
print(len(df_collab_subset))
df_collab_subset['application_year'] = df_collab_subset.apply(lambda x: df_ipgod101.application_year[int(x["australian_appl_no"])], axis=1)

4203


/Users/Goyder/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [14]:
df_collab_subset

,ipa_applt_id,australian_appl_no,appln_type,name,cleanname,country,australian,entity,ipa_id,abn,...,qg,state,postcode,patstat_appln_id,applicant_type,sa2_main11,sa2_name11,gcc_code11,gcc_name11,application_year
535,336619,2005253642,PAT,Grains Research and Development Corporation,GRAINS RESEARCH & DEVELOPMENT CORP,AU,1.0,1.0,86044.0,5.561122e+10,...,9.0,ACT,2600.0,2721471.0,Large firm,801061065.0,Kingston - Barton,8ACTE,Australian Capital Territory,2007
536,336618,2005253642,PAT,Agriculture Victoria Services Pty Ltd,AGRICULTURE VICTORIA SERVICES,AU,1.0,1.0,352822.0,2.300660e+10,...,8.0,VIC,3049.0,2721471.0,Large firm,210051245.0,Gladstone Park - Westmeadows,2GMEL,Greater Melbourne,2007
537,336620,2005253642,PAT,La Trobe University,LA TROBE UNIVERSITY,AU,1.0,1.0,142150.0,6.480474e+10,...,6.0,VIC,3083.0,2721471.0,Large firm,209011196.0,Bundoora - East,2GMEL,Greater Melbourne,2007
1142,7593989,2005263199,PAT,H-E PARTS INTERNATIONAL CRUSHING SOLUTIONS PTY...,HE PARTS INTL CRUSHING SOLUTIONS,AU,1.0,1.0,923727.0,3.300930e+10,...,7.0,WA,6165.0,2723009.0,SME,507031172.0,Hope Valley - Postans,5GPER,Greater Perth,2007
1143,338275,2005263199,PAT,Crushing & Mining Equipment Pty Ltd,CRUSHING & MINING EQUIPMENT,AU,1.0,1.0,923727.0,3.300930e+10,...,7.0,WA,6165.0,2723009.0,SME,507031172.0,Hope Valley - Postans,5GPER,Greater Perth,2007
1537,338842,2005266851,PAT,Orica Australia Pty. Ltd.,ORICA AUSTRALIA,AU,1.0,1.0,203584.0,9.900412e+10,...,8.0,VIC,3000.0,2723534.0,SME,206011106.0,Brunswick East,2GMEL,Greater Melbourne,2007
1538,6645205,2005266851,PAT,IXOM Operations Pty Ltd,IXOM OPERATIONS,AU,1.0,1.0,417901.0,5.160055e+10,...,9.0,VIC,3002.0,2723534.0,SME,206041119.0,East Melbourne,2GMEL,Greater Melbourne,2007
1879,339295,2005269278,PAT,Adline Group Pty Limited,ADLINE GROUP,AU,1.0,1.0,349867.0,8.705029e+10,...,9.0,NSW,2022.0,2723960.0,SME,118011341.0,Bondi Junction - Waverly,1GSYD,Greater Sydney,2007
1880,339296,2005269278,PAT,SouthCorp Wines Pty Ltd,SOUTHCORP WINES,AU,1.0,1.0,68236.0,7.400001e+10,...,8.0,NSW,2064.0,2723960.0,SME,121011401.0,St Leonards - Naremburn,1GSYD,Greater Sydney,2007
2169,7591518,2005270747,PAT,Antec Group Pty Ltd,ANTEC GROUP,AU,1.0,1.0,915972.0,1.700301e+10,...,8.0,NSW,2148.0,2724262.0,SME,116011303.0,Blacktown (East) - Kings Park,1GSYD,Greater Sydney,2007


In [15]:
list(json_data.keys())

['2015101098',
 '2007327562',
 '2008905564',
 '2011274302',
 '2008100652',
 '2015904875',
 '2015101037',
 '2009329832',
 '2007231821',
 '2008200431',
 '2012903673',
 '2010901455',
 '2016903946',
 '2011202494',
 '2009321523',
 '2008200558',
 '2009257187',
 '2010238563',
 '2008200634',
 '2010200996',
 '2008200939',
 '2013905016',
 '2007221735',
 '2009904881',
 '2014902306',
 '2014900763',
 '2013255094',
 '2010900597',
 '2010201727',
 '2013205231',
 '2014902310',
 '2012202007',
 '2008903042',
 '2013100288',
 '2015903334',
 '2008201858',
 '2015230873',
 '2013902219',
 '2015101279',
 '2011220342',
 '2013202517',
 '2010905287',
 '2012902719',
 '2011900861',
 '2012216392',
 '2016905122',
 '2015903329',
 '2007260584',
 '2010330676',
 '2008341009',
 '2008100267',
 '2012904455',
 '2015201156',
 '2015904794',
 '2007219063',
 '2013903012',
 '2012101072',
 '2015902467',
 '2008212054',
 '2010327312',
 '2012100744',
 '2009240784',
 '2010278661',
 '2007903674',
 '2012212390',
 '2007237197',
 '20102465

# WA Patents only - no longer used

For reference, but no longer used, we can extract the WA patents in a few ways. First, by postcode:

In [16]:
# Which patents were involved in WA?
df_wa_subset = df_collab_subset[df_collab_subset['postcode'] > 5999]
df_wa_subset = df_wa_subset[df_wa_subset['postcode'] < 7000]
print(df_wa_subset)

        ipa_applt_id  australian_appl_no appln_type  \
1142         7593989          2005263199        PAT   
1143          338275          2005263199        PAT   
3652          341155          2005279714        PAT   
3653         7609028          2005279714        PAT   
7347         7588557          2005301095        PAT   
7348          344861          2005301095        PAT   
9296          346766          2005312340        PAT   
9297         6807538          2005312340        PAT   
10471         347904          2005318937        PAT   
10472         347903          2005318937        PAT   
10473        5188006          2005318938        PAT   
10474         347905          2005318938        PAT   
12378         349946          2005333891        PAT   
15050         357847          2006216122        PAT   
15051         357849          2006216122        PAT   
15052         357846          2006216122        PAT   
15053         357848          2006216122        PAT   
18072     

In [17]:
patent_ids = df_collab_subset['australian_appl_no'].unique()
patent_dict = dict()
for patent_id in patent_ids:
    patent_dict[patent_id] = dict()
    patent_dict[patent_id]["year"] = df_collab_subset[df_collab_subset["australian_appl_no"] == patent_id]["application_year"].values[0]
print(patent_dict)
print(len(patent_dict))

{2012901378: {'year': 2012}, 2007904259: {'year': 2007}, 2010210310: {'year': 2011}, 2010210315: {'year': 2011}, 2013904917: {'year': 2013}, 2011238422: {'year': 2012}, 2010341404: {'year': 2012}, 2011238431: {'year': 2012}, 2015100960: {'year': 2015}, 2013327393: {'year': 2015}, 2015900567: {'year': 2015}, 2008903715: {'year': 2008}, 2010902565: {'year': 2010}, 2009202012: {'year': 2009}, 2012266538: {'year': 2012}, 2011250731: {'year': 2011}, 2005299245: {'year': 2007}, 2014900763: {'year': 2014}, 2013900850: {'year': 2013}, 2009321523: {'year': 2011}, 2010230836: {'year': 2011}, 2009903158: {'year': 2009}, 2015100983: {'year': 2015}, 2015100984: {'year': 2015}, 2012258361: {'year': 2012}, 2013900858: {'year': 2013}, 2009202749: {'year': 2009}, 2008100928: {'year': 2008}, 2015100994: {'year': 2015}, 2015100995: {'year': 2015}, 2015100996: {'year': 2015}, 2015100997: {'year': 2015}, 2009100360: {'year': 2009}, 2013900876: {'year': 2013}, 2013900877: {'year': 2013}, 2015200710: {'year'

In [18]:
len(df_collab_subset[df_collab_subset['state'] == 'WA'])

334

In [19]:
df_collab_subset[df_collab_subset['state'] == 'WA']['australian_appl_no'].unique()

array([2005263199, 2005279714, 2005301095, 2005312340, 2005318937,
       2005318938, 2005333891, 2006216122, 2006235205, 2006238316,
       2006284523, 2006294417, 2006308519, 2006317525, 2007200252,
       2007203730, 2007231544, 2007231552, 2007266335, 2007312928,
       2007900541, 2007903607, 2007904570, 2007904655, 2007905170,
       2007906896, 2008100563, 2008200431, 2008202719, 2008207517,
       2008207519, 2008251025, 2008267751, 2008303050, 2008316312,
       2008324749, 2008902804, 2008903042, 2008903418, 2008905126,
       2008905159, 2009100910, 2009208073, 2009227977, 2009233523,
       2009233524, 2009257202, 2009257204, 2009900134, 2009901625,
       2009901627, 2009903016, 2009903618, 2009904115, 2009904773,
       2009904829, 2010100941, 2010213361, 2010214681, 2010217183,
       2010283957, 2010292969, 2010901633, 2010901634, 2010902191,
       2010902461, 2010902565, 2010904249, 2010905277, 2010905619,
       2011101231, 2011101686, 2011200815, 2011217747, 2011223

# Collaboration Metrics and Bonus Points

We want to grant:

1. Points for collaboration between disparate companies
2. Points for collaboration between different state organisations

### Points then feed in for each patent application, and then into an SA2. 

### 1. Points for collaboration between disparate size companies

In [20]:
# Test for 1...
for patent_id in (patent_dict).keys():
    print(patent_id)
    applicant_types = (list(df_ipgod102[df_ipgod102['australian_appl_no'] == patent_id]['applicant_type'].unique()))
    if len(applicant_types) > 1:
        collaboration_occurred = True
    else:
        collaboration_occurred = False
    print("Collaboration between disparate size organisations occurred for {0}: {1}".format(patent_id, collaboration_occurred))
    patent_dict[patent_id]['factor_disparate_size'] = collaboration_occurred

2012901378
Collaboration between disparate size organisations occurred for 2012901378: True
2007904259
Collaboration between disparate size organisations occurred for 2007904259: True
2010210310
Collaboration between disparate size organisations occurred for 2010210310: True
2010210315
Collaboration between disparate size organisations occurred for 2010210315: True
2013904917
Collaboration between disparate size organisations occurred for 2013904917: True
2011238422
Collaboration between disparate size organisations occurred for 2011238422: True
2010341404
Collaboration between disparate size organisations occurred for 2010341404: True
2011238431
Collaboration between disparate size organisations occurred for 2011238431: False
2015100960
Collaboration between disparate size organisations occurred for 2015100960: False
2013327393
Collaboration between disparate size organisations occurred for 2013327393: True
2015900567
Collaboration between disparate size organisations occurred for 201

In [21]:
print(patent_dict)

{2012901378: {'factor_disparate_size': True, 'year': 2012}, 2007904259: {'factor_disparate_size': True, 'year': 2007}, 2010210310: {'factor_disparate_size': True, 'year': 2011}, 2010210315: {'factor_disparate_size': True, 'year': 2011}, 2013904917: {'factor_disparate_size': True, 'year': 2013}, 2011238422: {'factor_disparate_size': True, 'year': 2012}, 2010341404: {'factor_disparate_size': True, 'year': 2012}, 2011238431: {'factor_disparate_size': False, 'year': 2012}, 2015100960: {'factor_disparate_size': False, 'year': 2015}, 2013327393: {'factor_disparate_size': True, 'year': 2015}, 2015900567: {'factor_disparate_size': False, 'year': 2015}, 2008903715: {'factor_disparate_size': True, 'year': 2008}, 2010902565: {'factor_disparate_size': True, 'year': 2010}, 2009202012: {'factor_disparate_size': False, 'year': 2009}, 2012266538: {'factor_disparate_size': True, 'year': 2012}, 2011250731: {'factor_disparate_size': False, 'year': 2011}, 2005299245: {'factor_disparate_size': False, 'year

In [22]:
# Random check...
df_ipgod102[df_ipgod102['australian_appl_no'] == 2007200252]["applicant_type"]

37615    Large firm
37616           SME
Name: applicant_type, dtype: object

In [23]:
# That's a pass!

# Points for collaboration with different states

In [24]:
# Test for 1...
for patent_id in (patent_dict).keys():
    print(patent_id)
    states = (list(df_ipgod102[df_ipgod102['australian_appl_no'] == patent_id]['state'].unique()))
    print(states)
    if len(states) > 1:
        collaboration_occurred = True
    else:
        collaboration_occurred = False
    print("Collaboration between disparate size organisations occurred for {0}: {1}".format(patent_id, collaboration_occurred))
    patent_dict[patent_id]['factor_collaboration_state'] = collaboration_occurred

2012901378
['VIC']
Collaboration between disparate size organisations occurred for 2012901378: False
2007904259
['VIC']
Collaboration between disparate size organisations occurred for 2007904259: False
2010210310
['QLD']
Collaboration between disparate size organisations occurred for 2010210310: False
2010210315
['VIC']
Collaboration between disparate size organisations occurred for 2010210315: False
2013904917
['NSW']
Collaboration between disparate size organisations occurred for 2013904917: False
2011238422
['VIC']
Collaboration between disparate size organisations occurred for 2011238422: False
2010341404
['VIC', 'NSW']
Collaboration between disparate size organisations occurred for 2010341404: True
2011238431
['VIC']
Collaboration between disparate size organisations occurred for 2011238431: False
2015100960
['VIC']
Collaboration between disparate size organisations occurred for 2015100960: False
2013327393
['NSW', nan]
Collaboration between disparate size organisations occurred f

Check again!

In [25]:
print(patent_dict)

{2012901378: {'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2012}, 2007904259: {'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2007}, 2010210310: {'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2011}, 2010210315: {'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2011}, 2013904917: {'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2013}, 2011238422: {'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2012}, 2010341404: {'factor_disparate_size': True, 'factor_collaboration_state': True, 'year': 2012}, 2011238431: {'factor_disparate_size': False, 'factor_collaboration_state': False, 'year': 2012}, 2015100960: {'factor_disparate_size': False, 'factor_collaboration_state': False, 'year': 2015}, 2013327393: {'factor_disparate_size': True, 'factor_collaboration_state': True, 'year': 2015}, 2015900567: {'factor_disparate_size': F

In [26]:
# Random check...
print(patent_dict[2011286161])
df_ipgod102[df_ipgod102['australian_appl_no'] == 2011286161]

{'factor_disparate_size': False, 'factor_collaboration_state': True, 'year': 2013}


,ipa_applt_id,australian_appl_no,appln_type,name,cleanname,country,australian,entity,ipa_id,abn,...,lon,qg,state,postcode,patstat_appln_id,applicant_type,sa2_main11,sa2_name11,gcc_code11,gcc_name11
204683,512109,2011286161,PAT,Jorrocks Pty Ltd.,JORROCKS,AU,1.0,1.0,129074.0,NaN,...,143.14999,4.0,VIC,3260.0,405012737.0,SME,217021424.0,Camperdown,2RVIC,Rest of Vic.
204684,5192113,2011286161,PAT,Samson Biotics Ltd,SAMSON BIOTICS,AU,1.0,1.0,46676.0,1.660350e+10,...,115.86400,9.0,WA,6000.0,405012737.0,SME,503021041.0,Perth City,5GPER,Greater Perth


In [27]:
# Random check...
print(patent_dict[2013904375])
df_ipgod102[df_ipgod102['australian_appl_no'] == 2013904375]

{'factor_disparate_size': True, 'factor_collaboration_state': False, 'year': 2013}


,ipa_applt_id,australian_appl_no,appln_type,name,cleanname,country,australian,entity,ipa_id,abn,...,lon,qg,state,postcode,patstat_appln_id,applicant_type,sa2_main11,sa2_name11,gcc_code11,gcc_name11
302420,6904371,2013904375,PAT,NON-ENTITY 208763,NON-ENTITY 208763,AU,1.0,NaN,533343.0,NaN,...,115.902,9.0,WA,6051.0,NaN,Private applicant,504011046.0,Maylands,5GPER,Greater Perth
302421,6904368,2013904375,PAT,NON-ENTITY 399349,NON-ENTITY 399349,AU,1.0,NaN,860104.0,NaN,...,115.820,8.0,WA,6065.0,NaN,Private applicant,505031108.0,Wanneroo,5GPER,Greater Perth
302422,6904372,2013904375,PAT,Falconcrest Holdings Pty Ltd,FALCONCREST HOLDINGS,AU,1.0,1.0,333914.0,4.800934e+10,...,115.839,8.0,WA,6065.0,NaN,SME,505031103.0,Madeley - Darch - Landsdale,5GPER,Greater Perth
302423,6904373,2013904375,PAT,NON-ENTITY 596968,NON-ENTITY 596968,AU,1.0,NaN,753507.0,NaN,...,115.892,8.0,WA,6051.0,NaN,Private applicant,504011046.0,Maylands,5GPER,Greater Perth
302424,6904369,2013904375,PAT,NON-ENTITY 730205,NON-ENTITY 730205,AU,1.0,NaN,814040.0,NaN,...,115.892,8.0,WA,6051.0,NaN,Private applicant,504011046.0,Maylands,5GPER,Greater Perth
302425,6904374,2013904375,PAT,Elsegood Holdings Pty Ltd,ELSEGOOD HOLDINGS,AU,1.0,1.0,323237.0,8.706005e+10,...,115.839,8.0,WA,6065.0,NaN,SME,505031103.0,Madeley - Darch - Landsdale,5GPER,Greater Perth
302426,6904370,2013904375,PAT,NON-ENTITY 409700,NON-ENTITY 409700,AU,1.0,NaN,814277.0,NaN,...,115.932,8.0,WA,6053.0,NaN,Private applicant,504011045.0,Bayswater - Embleton - Bedford,5GPER,Greater Perth


That's a pass on both random checks!

# Turning these factors into additional collab points

We can have a list of patent IDs that are marked with either state or size collaboration.
The trick is now to turn this into additional points to be adjusted, on the basis of SA2 regions.

1. Each patent ID has a host of SA2 locations. For simplicity, we could just work through each row in each patent ID and just add points to all the SA2s involved...

In [28]:
df_output = pd.DataFrame()

for year in list(df_collab_subset['application_year'].unique()):
    print(year)
    
    sa2_dict = {}
    for patent_id in list(patent_dict):
        print(patent_dict[patent_id]["year"])
        if patent_dict[patent_id]["year"] != year:
            continue
            
        bonus_score = 0
        print("State collaboration:" + str(patent_dict[patent_id]['factor_collaboration_state']))
        if patent_dict[patent_id]['factor_collaboration_state']:
            bonus_score += 1
        print("Size collaboration:" + str(patent_dict[patent_id]['factor_disparate_size']))
        if patent_dict[patent_id]['factor_disparate_size']:
            bonus_score += 1
        print(bonus_score)
        
        for sa2 in df_ipgod102[df_ipgod102['australian_appl_no'] == patent_id]['sa2_main11'].values:
            if sa2 in sa2_dict:
                sa2_dict[sa2] += bonus_score
            else:
                sa2_dict[sa2] = bonus_score
            
    df_sa2_scores = pd.DataFrame(list(sa2_dict.items()))
    df_sa2_scores["year"] = year
    
    df_output = df_output.append(df_sa2_scores)
        
print(sa2_dict)

2007
2012
2007
State collaboration:False
Size collaboration:True
1
2011
2011
2013
2012
2012
2012
2015
2015
2015
2008
2010
2009
2012
2011
2007
State collaboration:False
Size collaboration:False
0
2014
2013
2011
2011
2009
2015
2015
2012
2013
2009
2008
2015
2015
2015
2015
2009
2013
2013
2015
2014
2008
2008
2009
2013
2014
2009
2014
2014
2011
2007
State collaboration:False
Size collaboration:False
0
2014
2015
2015
2015
2013
2012
2014
2007
State collaboration:True
Size collaboration:True
2
2007
State collaboration:True
Size collaboration:False
1
2013
2013
2008
2015
2007
State collaboration:False
Size collaboration:False
0
2009
2015
2014
2012
2013
2013
2010
2015
2009
2013
2015
2013
2015
2013
2008
2008
2016
2011
2009
2009
2007
State collaboration:False
Size collaboration:False
0
2011
2015
2015
2015
2012
2015
2007
State collaboration:True
Size collaboration:True
2
2014
2015
2013
2010
2010
2014
2008
2008
2011
2012
2008
2010
2013
2015
2015
2015
2015
2015
2010
2013
2014
2015
2012
2007
State collab

Let's clean up the output.

In [29]:
print(df_output.dropna())

              0   1  year
2   210051243.0   0  2007
4   121041416.0   0  2007
5   121041417.0  16  2007
6   125011473.0   0  2007
7   506031124.0   1  2007
8   119011356.0   0  2007
9   403031069.0   3  2007
11  128021538.0   2  2007
12  206041123.0   0  2007
13  206041124.0   4  2007
14  206041125.0   1  2007
15  206041126.0   4  2007
16  126021501.0   2  2007
17  122021423.0   1  2007
18  103041077.0   3  2007
19  301011002.0   2  2007
20  125031486.0   4  2007
23  206061136.0   0  2007
24  206061137.0   0  2007
25  206061138.0   1  2007
26  401031011.0   0  2007
27  210011228.0   1  2007
28  315021406.0   0  2007
29  212041311.0   0  2007
30  212041314.0   4  2007
32  201011003.0   0  2007
33  506071147.0   4  2007
34  105031101.0   0  2007
36  120011383.0   1  2007
38  111031231.0   5  2007
..          ...  ..   ...
58  117031338.0   0  2016
59  801051052.0   2  2016
60  503021042.0   4  2016
61  309081262.0   2  2016
62  401071024.0   0  2016
63  507051188.0   0  2016
64  10202104

In [30]:
df_output = df_output.dropna()
df_output[0] = df_output[0].apply(int)
df_output.to_csv("data_output/collab_points.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data_output/collab_points.csv'

In [ ]:
df_output

Repeating the above, but sans-years:

In [31]:
df_output_yearless = pd.DataFrame()

sa2_dict_yearless = {}
for patent_id in list(patent_dict):

    bonus_score = 0
    print("State collaboration:" + str(patent_dict[patent_id]['factor_collaboration_state']))
    if patent_dict[patent_id]['factor_collaboration_state']:
        bonus_score += 1
    print("Size collaboration:" + str(patent_dict[patent_id]['factor_disparate_size']))
    if patent_dict[patent_id]['factor_disparate_size']:
        bonus_score += 1
    print(bonus_score)

    for sa2 in df_ipgod102[df_ipgod102['australian_appl_no'] == patent_id]['sa2_main11'].values:
        if sa2 in sa2_dict_yearless:
            sa2_dict_yearless[sa2] += bonus_score
        else:
            sa2_dict_yearless[sa2] = bonus_score

df_sa2_scores_yearless = pd.DataFrame(list(sa2_dict_yearless.items()))



State collaboration:False
Size collaboration:True
1
State collaboration:False
Size collaboration:True
1
State collaboration:False
Size collaboration:True
1
State collaboration:False
Size collaboration:True
1
State collaboration:False
Size collaboration:True
1
State collaboration:False
Size collaboration:True
1
State collaboration:True
Size collaboration:True
2
State collaboration:False
Size collaboration:False
0
State collaboration:False
Size collaboration:False
0
State collaboration:True
Size collaboration:True
2
State collaboration:False
Size collaboration:False
0
State collaboration:False
Size collaboration:True
1
State collaboration:True
Size collaboration:True
2
State collaboration:True
Size collaboration:False
1
State collaboration:False
Size collaboration:True
1
State collaboration:False
Size collaboration:False
0
State collaboration:False
Size collaboration:False
0
State collaboration:False
Size collaboration:False
0
State collaboration:False
Size collaboration:False
0
State co

In [32]:
df_sa2_scores_yearless = df_sa2_scores_yearless.dropna()
df_sa2_scores_yearless[0] = df_sa2_scores_yearless[0].apply(int)

In [33]:
df_sa2_scores_yearless

,0,1
2,303021058,0
3,303021059,1
4,311061334,0
6,303031065,0
7,317011458,0
8,124051471,0
9,506031124,7
10,115041301,0
11,506031126,0
13,506031129,0


In [34]:
df_sa2_scores_yearless.to_csv("output_data/collab_points_yearless.csv")